# Retrieve Catalog Data from `catalogue.json`

The file `catalogue.json` contains all the catalog data for an [ORACC](http://oracc.org) project (for general information, see the [Oracc Open Data](http://oracc.org/doc/opendata) page). The `zip` that contains all JSON files of a particular project can be found at `http://build-oracc.museum.upenn.edu/json/[PROJECT].zip`. In the URL replace [PROJECT] with your project or sub-project name (e.g. `dcclt` or `cams/gkab`).

The main node in a `catalogue.json` file is called `members`. This node contains the information of all the fields and all the entries in the project catalog.

In [1]:
import pandas as pd
import zipfile
import json
import requests
import errno
import os
import tqdm

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Name
We will download and manipulate one catalog file at a time. Different [ORACC](http://oracc.org) projects use different fields in their catalogs, the catalogs are not mutually compatible.

Provide a project name. Note that subprojects must be processed separately, they are not included in the main project. A subproject is named `[PROJECT]/[SUBPROJECT]`, for instance `saao/saa01`.

In [3]:
projects = input('Project(s): ').lower().strip() # lowercase user input and strip accidental spaces

Project(s): fout, blms, saao/saa17


## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [4]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.2 Download the ZIP files
Download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `[PROJECT].zip` (for instance: `dcclt.zip`). For subprojects the file is called `[PROJECT-SUBPROJECT].zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that. For the chunking code see [this page](https://www.smallsurething.com/how-to-read-a-file-properly-in-python/).

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section ...

In [5]:
non_existent = []
CHUNK = 16 * 1024
for project in tqdm.tqdm(p):
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    r = requests.get(url)
    if r.status_code == 200:
        print("Downloading " + url + " saving as " + file)
        with open(file, 'wb') as f:
            for c in r.iter_content(chunk_size=CHUNK):
                f.write(c)
    else:
        print(url + " does not exist.")
        non_existent.append(project)
p = [i for i in p if i not in non_existent] # remove non-existing project names from list

  0%|                                                    | 0/3 [00:00<?, ?it/s]

http://build-oracc.museum.upenn.edu/json/fout.zip does not exist.


 33%|██████████████▋                             | 1/3 [00:00<00:00,  2.95it/s]

 67%|█████████████████████████████▎              | 2/3 [00:05<00:01,  1.70s/it]

100%|████████████████████████████████████████████| 3/3 [00:07<00:00,  2.01s/it]


## 2 Extract Catalogue Data from `JSON` files
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file, named `[PROJECT].zip` is located in the directory `jsonzip`. Each of these `zip` files includes a file called `catalogue.json`. This file is extracted and read with the command `json.loads()`, which reads the json data and transforms it into a JSON object - a sequence of names and values.

The JSON object is transformed into a Pandas Dataframe. The dataframe needs to be transposed (`.T`), so that the P, Q, and X numbers become indexes or row names (rather than column names), and each column represents a field in the catalog.  The individual dataframes (one for each project requested) are concatenated. Since individual [ORACC](http://oracc.org) project catalogs may have different fields, the dataframes may have different column names. By default Pandas concatenation uses an `outer join` so that all column names of all the catalogs are preserved.

In [7]:
df = pd.DataFrame() # create an empty dataframe
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    try:
        st = z.read(project + '/catalogue.json').decode('utf-8')  #read and decode the catalogue.json file of one project
                                                                # the result is a string object
    except:
        print(project + '/catalogue.json' + ' is not available or not complete')
        continue
    cat = json.loads(st)
    cat = cat['members']  # select the 'members' node 
    for item in cat.values():
        item["project"] = project # add project name as separate field
    cat_df = pd.DataFrame(cat).T
    df = pd.concat([df, cat_df])

## Create Dataframe
 Creating a Dataframe is not necessary, one may also manipulate the dictionary directly, but for demonstration purposes the Dataframe is a handy format. In manipulating the dictionary directly it is important to keep in mind that not all catalog fields have data for all entries, which means that not all dictionary keys are available for each P, Q, or X number.

The function `fillna('')` will put a blank (instead of `NaN`) in all fields that have no entry.

Example code for slicing the dictionary to select all entries that have `provenience = 'Ur'`:
> `urcat = {key:value for key, value in d.items() if 'provenience' in d[key] and d[key]['provenience'] == 'Ur'}

In [8]:
df = df.fillna('')
df

,accession_no,acquisition_history,ancient_author,ark,ark_number,atf_source,atf_up,author,bibliography__book_title,bibliography__id_biblio,...,title,trans,translit_ed,uploaded,uri,user,vol_title,volume,width,xproject
P223387,,,,,,,,"van Dijk, Johannes J. A.",,,...,,,,,http://cdli.ucla.edu/P223387,,,,,CDLI
P223392,,,,21198/zz001s7hkd,,,,"van Dijk, Johannes J. A.",,,...,,[en],,,http://cdli.ucla.edu/P223392,,,,?,CDLI
P223434,,,,,,,,"van Dijk, Johannes J. A.",,,...,,,,,http://cdli.ucla.edu/P223434,,,,,CDLI
P223478,,,,21198/zz001s91nk,,,,"van Dijk, Johannes J. A.",,,...,,[en],,,http://cdli.ucla.edu/P223478,,,,?,CDLI
P231196,,,,21198/zz001rcjtw,,,,,,,...,,,,,http://cdli.ucla.edu/P231196,,,,?,CDLI
P236940,"1880-07-19, 0160",,,,,,,,,,...,,,,,http://cdli.ucla.edu/P236940,,,,,CDLI
P237029,"1881-02-04, 0440",,,,,,,"Geller, Markham J.",,,...,,,,,http://cdli.ucla.edu/P237029,,,,,CDLI
P237069,K 21079,,,,,,,,,,...,,,,,http://cdli.ucla.edu/P237069,,,,,CDLI
P237082,"1881-07-27, 075",,,,,,,"Geller, Markham J.",,,...,,,,,http://cdli.ucla.edu/P237082,,,,,CDLI
P237100,"1881-07-27, 0261",,,,,,,,,,...,,,,,http://cdli.ucla.edu/P237100,,,,,CDLI


## Select Relevant Fields
[ORACC](http://oracc.org) catalogs may have custom fields, the only fields that are obligatory are `id_text` (the P, Q, or X number that identifies the text, for instance "P243546") and `designation` (the human-readable reference, for instance "VS 17, 012"). In order to find out which fields are available one may list the column names of the DataFrame.

First display all available fields, then select the ones that are relevant for the task at hand. 

In [9]:
df.columns

Index(['accession_no', 'acquisition_history', 'ancient_author', 'ark',
       'ark_number', 'atf_source', 'atf_up', 'author',
       'bibliography__book_title', 'bibliography__id_biblio',
       'bibliography__journal_title', 'bibliography__shortref',
       'bibliography__volume_number', 'ch_name', 'ch_no', 'ch_num_name',
       'checked', 'collection', 'collection_copyright',
       'condition_description', 'credits', 'date_entered', 'date_of_origin',
       'date_remarks', 'date_updated', 'dates_referenced', 'db_source',
       'designation', 'display_name', 'editor', 'excavation_no', 'external_id',
       'findspot_remarks', 'findspot_square', 'funder', 'genre',
       'google_earth_collection', 'height', 'id', 'id_text', 'id_text_int',
       'images', 'join_information', 'langs', 'language', 'language_remarks',
       'last_mod_by', 'last_modified', 'lemmed', 'lineart_up', 'material',
       'museum_no', 'object_preservation', 'object_remarks', 'object_type',
       'owner', 'per

In [12]:
df1 = df[['designation', 'period', 'provenience',
        'museum_no', 'project']]
df1

,designation,period,provenience,museum_no,project
P223387,"TIM 09, 056 (IM 003225)",Old Babylonian,Assur,IM 003225,blms
P223392,"TIM 09, 057 (IM 003263)",Neo-Babylonian,uncertain,IM 003263,blms
P223434,"TIM 09, 062",Old Babylonian,uncertain,"IM 021180,01",blms
P223478,"TIM 09, 032 (IM 058080)",Hellenistic,Uruk,IM 058080,blms
P231196,N 4950,Neo-Babylonian,Nippur,N 4950,blms
P236940,"1880-07-19, 0160",Neo-Assyrian,Nineveh,,blms
P237029,"1881-02-04, 0440",Neo-Assyrian,Nineveh,,blms
P237069,K 21079,Neo-Assyrian,Nineveh,,blms
P237082,"1881-07-27, 075",Neo-Assyrian,Nineveh,,blms
P237100,"1881-07-27, 0261",Neo-Assyrian,Nineveh,,blms


## Manipulate
The Dataframe may now be manipulated with standard Pandas methods. The example code selects the texts from Ur.
> `ur = df1[df1.provenience == "Ur"]`

## Save
Save the resulting data set as a `csv` file. `UTF-8` encoding is the encoding with the widest support in text analysis (and also the encoding used by [ORACC](http://oracc.org)). If you intend to use the catalog file in Excel, however, it is better to use `utf-16` encoding.

In [11]:
filename = 'catalog.csv'
with open('output/' + filename, 'w', encoding='utf-8') as w:
    df1.to_csv(w)